In [1]:
from string import Template
from pathlib import Path
from langchain_experimental.tools.python.tool import PythonAstREPLTool
import pandas as pd
from tools import tools

template = Template(Path(f'templates/df_react_agent.txt').read_text())


df = pd.read_excel('leave.xlsx')
df_head = df.head().to_markdown()
print(df_head)

python_repl = PythonAstREPLTool(locals={'df':df})

def plot_repl(tool_input):
    python_repl.run(tool_input)
    return '图片已生成'

tools_description = tools['description']

tools_map ={
    'dataframe_repl': python_repl.run,
    'plot_repl': plot_repl
}

|    | 申请人   | 申请人部门   |         申请人账号 | 当前审批状态   |      证件号码 |        手机 | 类别   | 开始日期   | 结束日期   |   开始时间 |   结束时间 | 开始时段   | 结束时段   |   请假时长 |   请假天数 | 请假原因                       |
|---:|:---------|:-------------|-------------------:|:---------------|--------------:|------------:|:-------|:-----------|:-----------|-----------:|-----------:|:-----------|:-----------|-----------:|-----------:|:-------------------------------|
|  0 | 丁娜     | 信息技术部   | 320723198911214629 | 已通过         |   3.20723e+17 | 1.59518e+10 | 年假   | 2023-01-28 | 2023-01-29 |        nan |        nan | 上午       | 下午       |        nan |          2 | 年假                           |
|  1 | 丁娜     | 信息技术部   | 320723198911214629 | 已通过         |   3.20723e+17 | 1.59518e+10 | 年假   | 2023-04-28 | 2023-04-28 |        nan |        nan | 上午       | 下午       |        nan |          1 | 回老家申请休年假一天           |
|  2 | 丁娜     | 信息技术部   | 320723198911214629 | 已通过         |   3.20723e+17 | 1.59518e+10 | 年假   | 2023-06-25 | 2023-06-

In [2]:
import re
import json
from model import client

# 将回复转为字典
def json_loads(response):
    pattern = re.compile(r'```json\n(.*)\n```', re.DOTALL)
    match = pattern.search(response)
    return json.loads(match.group(1))

# 生成prompt
def generate_prompt(query=''):
    return template.substitute(query=query, df_head=df_head, tools_description=tools_description)


# 调用大语言模型
def chat_with_llm(messages):
    response = client.chat.completions.create(
        model='glm-4-plus',
        messages=messages,
        temperature=0,
        max_tokens=4096,
    ).choices[0].message.content

    response = json_loads(response)

    return response

# 解析响应
def parse_thoughts(response):
    try:
        thoughts = response.get('thoughts')
        observation = response.get('observation')
 

        plan = thoughts.get('plan')
        criticism = thoughts.get('criticism')
        speak = thoughts.get('speak')
        reasoning = thoughts.get('reasoning')

        prompt = f" plan: {plan} \n criticism: {criticism} \n reasoning: {reasoning} \n observation: {observation} \n speak: {speak}"
        print(f'\033[1;35m Thoughts: \n{prompt} \033[0m')

        return prompt
    except Exception as e:
        print(f'\033[1;31m Error: {e} \033[0m')
        return str(e)
    

# agent执行
def agent_execute(query, max_request_time=10):
    cur_request_time = 0

    # 第一次请求prompt
    prompt = generate_prompt(query)
    messages = [
        {"role": "user", "content": prompt}
    ]


    while cur_request_time < max_request_time:
        cur_request_time += 1

        # 发送请求
        response = chat_with_llm(messages)

        # 解析思考
        thoughts = parse_thoughts(response)

        # 解析动作
        action = response.get('action')
        action_name = action.get('name')
        action_args = action.get('args')
        print(f'\033[1;32m Action: \n action_name: {action_name} \n action_args: {action_args} \033[0m')

        # 判断是否结束
        if action_name == 'finish':
            final_answer = action_args.get('answer')
            print(f' Final Answer: \n {final_answer}')
            break
        else:
            agent_scratch = f'action_name: {action_name}\naction_args: {action_args}\n'
            try:
                func = tools_map.get(action_name)
                func_result = func(**action_args)
                agent_scratch += f'action_result: {func_result}'
                print(f'\033[1;36m action_result: {func_result} \033[0m')
            except Exception as e:
                print(f'\033[1;31m Error: {e} \033[0m')
                agent_scratch += f'action_result: {e}'
            
            # 更新prompt
            messages = [
                {"role": "user", "content": f'{prompt}\n## agent_scratchpad:\n{thoughts}\n{agent_scratch}'}
            ]

In [7]:
agent_execute('请假最多的5个人是谁，请分别列出这些人的请假次数、最早假期开始时间、最早假期结束时间')

 Thoughts: 
 plan: 首先为每个请假记录添加一个'请假次数'列，然后按'申请人'分组，计算每个申请人的请假次数、最早假期开始时间和最早假期结束时间。最后按请假次数降序排序，取前5名。 
 criticism: 需要确保数据中没有重复记录，否则会影响请假次数的统计。 
 reasoning: 通过分组和聚合函数可以有效地统计每个申请人的请假情况，排序后可以找到请假最多的5个人。 
 observation: 正在执行数据处理操作，等待结果。 
 speak: 正在计算请假最多的5个人及其相关数据。 
 Action: 
 action_name: dataframe_repl 
 action_args: {'tool_input': "import pandas as pd\n\ndf['请假次数'] = 1\n\ngrouped_df = df.groupby('申请人').agg({'请假次数': 'sum', '开始日期': 'min', '结束日期': 'min'})\nsorted_df = grouped_df.sort_values(by='请假次数', ascending=False).head(5)\nprint(sorted_df)"} 
 action_result:      请假次数        开始日期        结束日期
申请人                              
张莹玺    30  2023-01-17  2023-01-17
陈怡舟    28  2022-12-09  2022-12-09
祖晓欢    25  2023-01-28  2023-01-28
徐凌     21  2022-12-06  2022-12-06
高萍     21  2022-12-13  2022-12-13
 
 Thoughts: 
 plan: 已成功计算每个申请人的请假次数、最早假期开始时间和最早假期结束时间，并按请假次数降序排序，取前5名。 
 criticism: 代码执行结果符合预期，但需要注意如果有新的数据加入，需要重新执行此代码以获取最新结果。 
 reasoning: 通过分组和聚合函数可以有效地统计每个申请人的请假情况，排序后可以找到请假最多的5个人，结果准确无误。 
 obser

In [3]:
agent_execute('用饼图展示请假次数前三的人的次数比例')

 Thoughts: 
 plan: 1. 使用value_counts()计算每个申请人的请假次数。2. 选择请假次数最多的前三人。3. 使用matplotlib绘制饼图，并设置中文字体。4. 保存饼图到当前目录。 
 criticism: 需要确保matplotlib库和simhei.ttf字体已正确安装和配置。 
 reasoning: 通过计算请假次数并绘制饼图，可以直观展示请假次数前三的人的次数比例。 
 observation: 正在生成饼图，任务即将完成。 
 speak: 正在生成请假次数前三的人的次数比例饼图，请稍等。 
 Action: 
 action_name: dataframe_repl 
 action_args: {'tool_input': "import matplotlib.pyplot as plt\nfrom matplotlib.font_manager import FontProperties\n\n# 计算请假次数\nleave_counts = df['申请人'].value_counts().head(3)\n\n# 设置中文字体\nfont = FontProperties(fname='simhei.ttf', size=14)\n\n# 绘制饼图\nplt.figure(figsize=(8, 6))\nplt.pie(leave_counts, labels=leave_counts.index, autopct='%1.1f%%', startangle=140, textprops={'fontproperties': font})\nplt.title('请假次数前三的人的次数比例', fontproperties=font)\nplt.savefig('plot.png')\nplt.close()"} 
 action_result:  
 Thoughts: 
 plan: 1. 使用value_counts()计算每个申请人的请假次数。2. 选择请假次数最多的前三人。3. 使用matplotlib绘制饼图，并设置中文字体。4. 保存饼图到当前目录。 
 criticism: 需要确保matplotlib库和simhei.ttf字体已正确安装和配置。 
 reasoning: 通过计算请假次数